# Install

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whic

In [2]:
!pip install spacy transformers google-cloud-bigquery
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 45.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Authentication of BigQuery and Drive

In [6]:
# # Authenticate Google Cloud for BigQuery
# from google.colab import auth
# auth.authenticate_user()

import os

secrets = {
  "google_application_credentials": "/content/customers-reviews-database-0aa166143b9b.json"
  }
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = secrets["google_application_credentials"]

project_id = "customers-reviews-database"
dataset_id = "amazon_db"


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Store results to bigQuery function

In [8]:
def save_to_bigquery(df, table_id):
    try:
        client = bigquery.Client()
        job = client.load_table_from_dataframe(df, table_id)
        job.result()
        print(f"Data uploaded to BigQuery table {table_id}.")
    except Exception as e:
        print(f"Error saving to BigQuery: {e}")

# Import

In [9]:
from datasets import load_dataset, Dataset
from google.cloud import bigquery
import pandas as pd
import os
import json
from google.cloud import bigquery
import pandas as pd
import spacy
from transformers import pipeline
from transformers import BertTokenizer



# Configurations

In [10]:
url = "McAuley-Lab/Amazon-Reviews-2023"
category = "Subscription_Boxes"
c = category.replace("_", "")
reviews_table_name = c + "Reviews"
products_table_name = c +"Products"
sentiments_table_name = c +"Sentiments"

# Extract Data

In [26]:
def get_data(url, category):

    reviews_dataset = load_dataset(url, f"raw_review_{category}", trust_remote_code=True)
    products_dataset = load_dataset(url, f"raw_meta_{category}", split="full", trust_remote_code=True)
    print("Data downloaded successfully!")
    return reviews_dataset, products_dataset

In [27]:
reviews_dataset, products_dataset = get_data(url, category)

Data downloaded successfully!


# Preprocess Data

In [28]:
def get_and_clean_data(url, category, reviews_dataset, products_dataset):

    reviews_data_frame = pd.DataFrame(reviews_dataset["full"].select_columns(["rating", "title", "text", "asin", "parent_asin", "helpful_vote", "timestamp", "verified_purchase"]))
    products_data_frame = pd.DataFrame(products_dataset.select_columns(["main_category", "title", "features", "price", "description", "parent_asin", "average_rating", "rating_number"]))

    print("get desired split of data")

    # Remove reviews with null or empty text
    reviews_data_frame = reviews_data_frame.dropna(subset=["text"])
    reviews_data_frame = reviews_data_frame[reviews_data_frame["text"].str.strip() != ""]
    print("Remove reviews with null or empty text")

    # Convert timestamp to datetime format
    reviews_data_frame["timestamp"] = pd.to_datetime(reviews_data_frame["timestamp"], errors="coerce",  unit='ms')
    print("Convert timestamp to datetime format")

    # # Convert 'features' column to string before dropping duplicates
    # products_data_frame['features'] = products_data_frame['features'].astype(str)

    # # remove duplicates
    # reviews_data_frame = reviews_data_frame.drop_duplicates()
    # products_data_frame = products_data_frame.drop_duplicates()
    # print("remove duplicates")

    # Fill missing helpful_vote with 0 (assuming it represents count)
    reviews_data_frame["helpful_vote"] = reviews_data_frame["helpful_vote"].fillna(0)
    print("Fill missing helpful_vote with 0 (assuming it represents count)")

    # Add ID column (auto increament)

    reviews_data_frame.insert(0, "id", range(1, len(reviews_data_frame) + 1))
    products_data_frame.insert(0, "id", range(1, len(products_data_frame) + 1))
    print("Add ID column (auto increament)")

    return reviews_data_frame, products_data_frame

Hugging face McAuley-Lab/Amazon-Reviews-2023 dataset

In [29]:
reviews_data_frame, products_data_frame = get_and_clean_data(url, category, reviews_dataset, products_dataset)

get desired split of data
Remove reviews with null or empty text
Convert timestamp to datetime format
Fill missing helpful_vote with 0 (assuming it represents count)
Add ID column (auto increament)


In [30]:
reviews_data_frame.head()

,id,rating,title,text,asin,parent_asin,helpful_vote,timestamp,verified_purchase
0,1,1.0,USELESS,Absolutely useless nonsense and a complete was...,B07G584SHG,B09WC47S3V,2,2020-10-08 05:10:57.705,True
1,2,2.0,Manufactured where?,"With a couple of the items, I wasn't quite sur...",B07QL1JRCN,B07QL1JRCN,20,2020-12-27 23:12:15.433,True
2,3,1.0,Little bang for your buck.,Two SMALL stuffed animals and 2 little bags of...,B07RBYJN37,B08N5QKX1Y,4,2021-01-06 12:48:35.319,True
3,4,5.0,New favorite box,"Although I don’t remember signing up for this,...",B07KM6T8GV,B07KM6T8GV,1,2021-03-19 12:19:11.887,True
4,5,5.0,Coctique,I loved every thing and could use it all. Thin...,B07NVL6TJG,B07NVKNVNM,0,2019-06-03 03:40:06.066,True


In [31]:
products_data_frame.head()

,id,main_category,title,features,price,description,parent_asin,average_rating,rating_number
0,1,SUBSCRIPTION BOXES,Loved Again Media - Movie Subscription Box - 1...,[10 gently used DVDs delivered to your door ev...,None,[],B08W5BSH6V,4.1,75
1,2,SUBSCRIPTION BOXES,Bravado Music Icons T-Shirt Club Subscription ...,[WHAT IS T-SHIRT CLUB? Created for the ultimat...,None,[],B08MV3QHL8,3.9,329
2,3,SUBSCRIPTION BOXES,Succulents Box - 2 Succulents 2 Clay Pots Subs...,[You'll receive a box of succulents in clay po...,None,[],B084KPB9K8,3.8,928
3,4,SUBSCRIPTION BOXES,The Legend of Vox Machina T-Shirt Club Subscri...,[WHAT IS T-SHIRT CLUB? Created for the ultimat...,None,[],B09YYGB4BH,4.4,47
4,5,SUBSCRIPTION BOXES,50 States Of Mine - Premium Subscription Box: ...,[6-10 quality curated items from a different s...,None,[],B092MLYDGR,4.6,6


In [32]:
save_to_bigquery(reviews_data_frame, f"{project_id}.{dataset_id}.{reviews_table_name}")
save_to_bigquery(products_data_frame, f"{project_id}.{dataset_id}.{products_table_name}")

Data uploaded to BigQuery table customers-reviews-database.amazon_db.SubscriptionBoxesReviews.


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: description.
  warnings.warn(


Data uploaded to BigQuery table customers-reviews-database.amazon_db.SubscriptionBoxesProducts.


In [33]:
selected_columns = ['id', 'text']
reviews_data = reviews_data_frame[selected_columns].copy()


In [34]:
reviews_data.head()

,id,text
0,1,Absolutely useless nonsense and a complete was...
1,2,"With a couple of the items, I wasn't quite sur..."
2,3,Two SMALL stuffed animals and 2 little bags of...
3,4,"Although I don’t remember signing up for this,..."
4,5,I loved every thing and could use it all. Thin...


# Processing Text

Summarize long reviews with BigBird Model

In [35]:

def summarize_texts(texts, max_length=400, limit=500):

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    summarizer = pipeline("summarization", model="google/bigbird-pegasus-large-arxiv", device=0)
    summaries = []
    i=0

    for text in texts:

        tokens = tokenizer.encode(text, add_special_tokens=True)  # Include special tokens for BERT
        original_length = len(tokens)

        if original_length > limit:
            summarized = summarizer([text], max_length = max_length, truncation=True)
            summaries.append(summarized[0]['summary_text'])


        else:
            # Skip very short reviews
            summaries.append(text)

        print(i)
        i+=1
    return summaries

Find the sentiment of reviews (NEGATIVE, POSITIVE) with Hugging face BERT model

In [36]:
def find_sentiment(model_name, texts):
    if model_name == "BERT-pretrained":

        sentiment_pipeline = pipeline('sentiment-analysis', device=0)
        sentiments = []
        for i, text in enumerate(texts):
            print(i)
            try:
                result = sentiment_pipeline(text)
                sentiments.append(result[0]['label'])
            except Exception as e:
                print(f"Error processing sentiment for text at index {i}: {e}")

                sentiments.append("Error")
        return sentiments

In [37]:
# Summarize the reviews
reviews_data['summary'] = summarize_texts(reviews_data['text'].tolist())
print("Summarization finished")
reviews_data.head()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

Input ids are automatically padded from 933 to 960 to be a multiple of `config.block_size`: 64
Attention type 'block_sparse' is not possible if sequence_length: 619 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Streaming output truncated to the last 5000 lines.
11206
11207
11208
11209
11210
11211
11212
11213
11214
11215
11216
11217
11218
11219
11220
11221
11222
11223
11224
11225
11226
11227
11228
11229
11230
11231
11232
11233
11234
11235
11236
11237
11238
11239
11240
11241
11242
11243
11244
11245
11246
11247
11248
11249
11250
11251
11252
11253
11254
11255
11256
11257
11258
11259
11260
11261
11262
11263
11264
11265
11266
11267
11268
11269
11270
11271
11272
11273
11274
11275
11276
11277
11278
11279
11280
11281
11282
11283
11284
11285
11286
11287
11288
11289
11290
11291
11292
11293
11294
11295
11296
11297
11298
11299
11300
11301
11302
11303
11304
11305
11306
11307
11308
11309
11310
11311
11312
11313
11314
11315
11316
11317
11318
11319
11320
11321
11322
11323
11324
11325
11326
11327
11328
11329
11330
11331
11332
11333
11334
11335
11336
11337
11338
11339
11340
11341
11342
11343
11344
11345
11346
11347
11348
11349
11350
11351
11352
11353
11354
11355
11356
11357
11358
11359
11360
11361
11362
11363
1

,id,text,summary
0,1,Absolutely useless nonsense and a complete was...,Absolutely useless nonsense and a complete was...
1,2,"With a couple of the items, I wasn't quite sur...","With a couple of the items, I wasn't quite sur..."
2,3,Two SMALL stuffed animals and 2 little bags of...,Two SMALL stuffed animals and 2 little bags of...
3,4,"Although I don’t remember signing up for this,...","Although I don’t remember signing up for this,..."
4,5,I loved every thing and could use it all. Thin...,I loved every thing and could use it all. Thin...


In [38]:
# Save DataFrame to CSV
file_path = '/content/drive/My Drive/summarized_reviews_df.csv'
reviews_data.to_csv(file_path, index=False)
print(f"DataFrame saved to {file_path}")

DataFrame saved to /content/drive/My Drive/summarized_reviews_df.csv


In [39]:
# Perform sentiment analysis
reviews_data['sentiment'] = find_sentiment("BERT-pretrained", reviews_data['summary'].tolist())
print("Sentiment analysis finished")
reviews_data.head()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


Streaming output truncated to the last 5000 lines.
11206
11207
11208
11209
11210
11211
11212
11213
11214
11215
11216
11217
11218
11219
11220
11221
11222
11223
11224
11225
11226
11227
11228
11229
11230
11231
11232
11233
11234
11235
11236
11237
11238
11239
11240
11241
11242
11243
11244
11245
11246
11247
11248
11249
11250
11251
11252
11253
11254
11255
11256
11257
11258
11259
11260
11261
11262
11263
11264
11265
11266
11267
11268
11269
11270
11271
11272
11273
11274
11275
11276
11277
11278
11279
11280
11281
11282
11283
11284
11285
11286
11287
11288
11289
11290
11291
11292
11293
11294
11295
11296
11297
11298
11299
11300
11301
11302
11303
11304
11305
11306
11307
11308
11309
11310
11311
11312
11313
11314
11315
11316
11317
11318
11319
11320
11321
11322
11323
11324
11325
11326
11327
11328
11329
11330
11331
11332
11333
11334
11335
11336
11337
11338
11339
11340
11341
11342
11343
11344
11345
11346
11347
11348
11349
11350
11351
11352
11353
11354
11355
11356
11357
11358
11359
11360
11361
11362
11363
1

,id,text,summary,sentiment
0,1,Absolutely useless nonsense and a complete was...,Absolutely useless nonsense and a complete was...,NEGATIVE
1,2,"With a couple of the items, I wasn't quite sur...","With a couple of the items, I wasn't quite sur...",NEGATIVE
2,3,Two SMALL stuffed animals and 2 little bags of...,Two SMALL stuffed animals and 2 little bags of...,NEGATIVE
3,4,"Although I don’t remember signing up for this,...","Although I don’t remember signing up for this,...",POSITIVE
4,5,I loved every thing and could use it all. Thin...,I loved every thing and could use it all. Thin...,POSITIVE


In [40]:
# Save DataFrame to CSV
file_path = '/content/drive/My Drive/complete_reviews_df.csv'
reviews_data.to_csv(file_path, index=False)
print(f"DataFrame saved to {file_path}")

DataFrame saved to /content/drive/My Drive/complete_reviews_df.csv


In [41]:
reviews_data = reviews_data.drop(columns=['text'])

# Store results to the bigquery database

In [42]:

save_to_bigquery(reviews_data, f"{project_id}.{dataset_id}.{sentiments_table_name}")

Data uploaded to BigQuery table customers-reviews-database.amazon_db.SubscriptionBoxesSentiments.
